<a href="https://colab.research.google.com/github/dhenypatungka/Utilities/blob/main/LipSyncHD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⛪ **LIP SYNC - AI**  

In [ ]:
# @markdown ## 🌄 1. Install LipSync
!git clone https://github.com/dhenypatungka/LipSyncHD.git
basePath = "/content/LipSyncHD"
%cd {basePath}

wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'
!gdown https://drive.google.com/uc?id=1QCFAzIFqC8DSGzM39yiCBVxupmXB0Wpn --output {wav2lipPath}'/checkpoints/'
!gdown https://drive.google.com/uc?id=1mYpbyUaVxb-sWogDX-6Ol1yBk0aVwHAC --output {wav2lipPath}'/checkpoints/'
!cd $gfpganFolderName && python setup.py develop
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P {gfpganFolderName}'/experiments/pretrained_models'

!pip install -r requirements.txt

In [ ]:
# @markdown ## 🌄 2. Download YouTube Video

import os
!pip install yt-dlp

#@markdown #### Find YouTube video
#@markdown ``https://youtu.be/twX1GZBoBmY`` ⛔ <br>
#@markdown ``https://www.youtube.com/watch?v=twX1GZBoBmY`` ✅

!rm -df youtube.mp4

#@markdown ___
from urllib import parse as urlparse
YOUTUBE_URL = 'https://www.youtube.com/watch?v=KfVdIKaQzW8' #@param {type:"string"}
url_data = urlparse.urlparse(YOUTUBE_URL)
query = urlparse.parse_qs(url_data.query)
YOUTUBE_ID = query["v"][0]

# remove previous input video
!rm -f /content/LipSyncHD/inputs/input_vid.mp4

#@markdown ___
#@markdown #### Trim (seconds)
start = 45 #@param {type:"integer"}
end = 55 #@param {type:"integer"}
interval = end - start

# Download the YouTube video using yt-dlp
!yt-dlp -f 'b[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

# Cut the video using FFmpeg
!ffmpeg -y -i youtube.mp4 -ss {start} -t {interval} -async 1 /content/LipSyncHD/inputs/input_vid.mp4

# Preview the trimmed video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/LipSyncHD/inputs/input_vid.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=600 controls><source src="{data_url}"></video>""")

In [ ]:
# @markdown ## 🌄 3. Upload Video From Your FIles
import os
import shutil
from google.colab import drive
from google.colab import files
from IPython.display import HTML, clear_output
from base64 import b64encode
import moviepy.editor as mp


def showVideo(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

def get_video_resolution(video_path):
    """Function to get the resolution of a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return (width, height)

def resize_video(video_path, new_resolution):
    """Function to resize a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    fourcc = int(video.get(cv2.CAP_PROP_FOURCC))
    fps = video.get(cv2.CAP_PROP_FPS)
    width, height = new_resolution
    output_path = os.path.splitext(video_path)[0] + '_720p.mp4'
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    while True:
        success, frame = video.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, new_resolution)
        writer.write(resized_frame)
    video.release()
    writer.release()

upload_method = "Upload"

# remove previous input video
if os.path.isfile('/content/LipSyncHD/inputs/input_vid.mp4'):
    os.remove('/content/LipSyncHD/inputs/input_vid.mp4')

if upload_method == "Upload":
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, '/content/LipSyncHD/inputs/input_vid.mp4')
    PATH_TO_YOUR_VIDEO = '/content/LipSyncHD/inputs/input_vid.mp4'


video_duration = mp.VideoFileClip(PATH_TO_YOUR_VIDEO).duration
if video_duration > 60:
    print("WARNING: Video duration exceeds 60 seconds. Please upload a shorter video.")
    raise SystemExit(0)

video_resolution = get_video_resolution(PATH_TO_YOUR_VIDEO)
print(f"Video resolution: {video_resolution}")
if video_resolution[0] >= 1920 or video_resolution[1] >= 1080:
    print("Resizing video to 720p...")
    os.system(f"ffmpeg -i {PATH_TO_YOUR_VIDEO} -vf scale=1280:720 /content/LipSyncHD/inputs/input_vid.mp4")
    PATH_TO_YOUR_VIDEO = "/content/LipSyncHD/inputs/input_vid.mp4"
    print("Video resized to 720p")
else:
    print("No resizing needed")

if upload_method == "Upload":
  clear_output()
  print("Input Video")
  #showVideo(PATH_TO_YOUR_VIDEO)
else:
    if os.path.isfile(PATH_TO_YOUR_VIDEO):
        shutil.copyfile(PATH_TO_YOUR_VIDEO, "/content/LipSyncHD/inputs/input_vid.mp4")
        print("Input Video")
        #showVideo(PATH_TO_YOUR_VIDEO)

#show generated video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/LipSyncHD/inputs/input_vid.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=600px height=auto autoplay loop controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
# @markdown ## 🌄 4. Upload Audio File
import os
from IPython.display import Audio
from IPython.core.display import display

upload_method = 'Upload'

#remove previous input audio
if os.path.isfile('/content/LipSyncHD/inputs/input_audio.mp3'):
    os.remove('/content/LipSyncHD/inputs/input_audio.mp3')

def displayAudio():
  display(Audio('/content/LipSyncHD/inputs/input_audio.mp3'))

#@markdown   Upload wav or mp3.

if upload_method == 'Upload':
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes.'.format(
        name=fn, length=len(uploaded[fn])))

  # Consider only the first file
  PATH_TO_YOUR_AUDIO = str(list(uploaded.keys())[0])

  # Load audio with specified sampling rate
  import librosa
  audio, sr = librosa.load(PATH_TO_YOUR_AUDIO, sr=None)

  # Save audio with specified sampling rate
  import soundfile as sf
  sf.write('/content/LipSyncHD/inputs/input_audio.mp3', audio, sr, format='mp3')

  displayAudio()

In [ ]:
# @markdown ## 🌄 5. Start LipSync
import os
outputPath = basePath+'/outputs'
inputAudioPath = basePath + '/inputs/input_audio.mp3'
inputVideoPath = basePath + '/inputs/input_vid.mp4'
lipSyncedOutputPath = basePath + '/outputs/result.mp4'

if not os.path.exists(outputPath):
  os.makedirs(outputPath)

!cd $wav2lipFolderName && python inference.py \
--checkpoint_path checkpoints/dp_gan.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath}

#show generated video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/LipSyncHD/outputs/result.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=600px height=auto controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
# @markdown ## 🌄 6. Extract Video To Frames
import cv2
from tqdm import tqdm
from os import path

import os

inputVideoPath = outputPath+'/result.mp4'
unProcessedFramesFolderPath = outputPath+'/frames'

if not os.path.exists(unProcessedFramesFolderPath):
  os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)


In [ ]:
# @markdown ## 🌄 7. Enhanced Images
!cd $gfpganFolderName && \
  python inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.3 -s 1 --bg_upsampler None

In [ ]:
# @markdown ## 🌄 8. Restore Images to Video
import os
restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

batch = 0
batchSize = 300
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end)
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)


  out = cv2.VideoWriter(processedVideoOutputPath+'/batch_'+str(batch).zfill(4)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
  batch = batch + 1

  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()


In [ ]:
# @markdown ## 🌄 9. Final Result
# @markdown Your video will download automatically, please wait a few minutes.
concatTextFilePath = outputPath + "/concat.txt"
concatTextFile=open(concatTextFilePath,"w")
for ips in range(batch):
  concatTextFile.write("file batch_" + str(ips).zfill(4) + ".avi\n")
concatTextFile.close()

concatedVideoOutputPath = outputPath + "/concated_output.avi"
!ffmpeg -y -f concat -i {concatTextFilePath} -c copy {concatedVideoOutputPath}

finalProcessedOuputVideo = processedVideoOutputPath + "/Final_Video.mp4"
!ffmpeg -y -i {concatedVideoOutputPath} -i {inputAudioPath} -map 0 -map 1:a -c:v copy -shortest {finalProcessedOuputVideo}

from google.colab import files
files.download(finalProcessedOuputVideo)

In [ ]:
# @markdown ## 🌄 8. Start Over & Cleaning
# @markdown Click this option if you want to start to create new Lipsync..
for f in os.listdir('/content/LipSyncHD/outputs/restored_imgs'):
  os.remove(os.path.join('/content/LipSyncHD/outputs/restored_imgs', f))
for f in os.listdir('/content/LipSyncHD/outputs/frames'):
  os.remove(os.path.join('/content/LipSyncHD/outputs/frames', f))
